<a href="https://colab.research.google.com/github/MiyokoPang/IR/blob/master/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Prerequisite

In [1]:
!git clone https://github.com/MiyokoPang/IR.git

Cloning into 'IR'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 24 (delta 5), reused 17 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (24/24), 871.37 KiB | 25.63 MiB/s, done.
Resolving deltas: 100% (5/5), done.
